# Similarity PDF Query Tutorial

In this tutorial, we demonstrate how to load a PDF and query it.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EvaDB server



In [1]:
from evadb.interfaces.relational.db import connect
conn = connect()
cursor = conn.cursor()

### Download PDFs

In [2]:
!wget -nc "https://www.dropbox.com/s/6jdcn33xizdtl9k/layout-parser-paper.pdf"
!wget -nc "https://www.dropbox.com/s/4q3bvne3m2vsu5g/state_of_the_union.pdf"

--2023-06-08 00:33:04--  https://www.dropbox.com/s/6jdcn33xizdtl9k/layout-parser-paper.pdf
Resolving www.dropbox.com (www.dropbox.com)... 

162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /s/raw/6jdcn33xizdtl9k/layout-parser-paper.pdf [following]
--2023-06-08 00:33:05--  https://www.dropbox.com/s/raw/6jdcn33xizdtl9k/layout-parser-paper.pdf
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com/cd/0/inline/B9ljG1NyIoayykdR-h689dkxr1xua5xJCK7aT6X0uVJc50H6cvf1z04-uy7rOZc_BsEETTG7ZUI1AudyeQabjEUHTF6knI39ScuEQZqDbTnrqzscJ_rCn1a2qvGH_fADzTvMukgb4NEzNlFKiPGkPr_6z6aYT8Y4tbrr0j8u412h1g/file# [following]
--2023-06-08 00:33:05--  https://ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com/cd/0/inline/B9ljG1NyIoayykdR-h689dkxr1xua5xJCK7aT6X0uVJc50H6cvf1z04-uy7rOZc_BsEETTG7ZUI1AudyeQabjEUHTF6knI39ScuEQZqDbTnrqzscJ_rCn1a2qvGH_fADzTvMukgb4NEzNlFKiPGkPr_6z6aYT8Y4tbrr0j8u412h1g/file
Resolving ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com (ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com)... 

162.125.9.15, 2620:100:601f:15::a27d:90f
Connecting to ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com (ucad57ff684b5dec838c0ffe9c84.dl.dropboxusercontent.com)|162.125.9.15|:443... 

...

302 Found
Location: https://uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com/cd/0/inline/B9kzLFZ_XndQE2e22FXXF6uNTdJcaMLCoRgfQ6Ryb3nDDLGomfgliN373FjtjjZu0Ds8_RtmkKbv_-LtPwJXFADku_3-Du-ppOo6FdmMdG_RAeXEdUHhk3fHi4ncMURPqmSsfGalNbubsxdZgR_S2NLlpJDCfLRxjIMlLSVa33H85w/file# [following]
--2023-06-08 00:33:07--  https://uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com/cd/0/inline/B9kzLFZ_XndQE2e22FXXF6uNTdJcaMLCoRgfQ6Ryb3nDDLGomfgliN373FjtjjZu0Ds8_RtmkKbv_-LtPwJXFADku_3-Du-ppOo6FdmMdG_RAeXEdUHhk3fHi4ncMURPqmSsfGalNbubsxdZgR_S2NLlpJDCfLRxjIMlLSVa33H85w/file
Resolving uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com (uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com)... 

162.125.9.15, 2620:100:601f:15::a27d:90f
Connecting to uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com (uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com)|162.125.9.15|:443... connected.


HTTP request sent, awaiting response... 

302 Found
Location: /cd/0/inline2/B9mcXWfidf_fGkv6M2zJhe23Q1EOdy2V66j0LncLjqXOEp-_ivoDBBCIjcM9aCzfZRGzLoclDRfqKwKtMpgzklZhWBnBSkeJ_-tzlIxMl_vzVF0ndBpYex1saFa6sv0koEYQ-dncISKy0S20vQZjyfiSeNWZbVdQRo7G-tDUUzhHXPO-zclC5sF9-TDrmigTt5Ak_mCY1h1dB060qe0NWB99Wjn6c6TGbmPrx6SySxhCKqShm2tJuGHpMF-b6mEOXdwtiBsmS-UPpc92C8KCmBzC4g5_w7A9T-_JqM6s_UqhcFwnE0rZG8GgciIxO-VWa552y9dTMQpNiVkyQ3Fmy5l4Ev6967rFtmqcwRTPKfFoiCyriNi_WddZhYzjyewNPcHyXvr9BDwKkRiR_gaqgB3tbl2GNsDEynERJTsTZtq8Ig/file [following]
--2023-06-08 00:33:07--  https://uc5501ef9872334ef2de339fb925.dl.dropboxusercontent.com/cd/0/inline2/B9mcXWfidf_fGkv6M2zJhe23Q1EOdy2V66j0LncLjqXOEp-_ivoDBBCIjcM9aCzfZRGzLoclDRfqKwKtMpgzklZhWBnBSkeJ_-tzlIxMl_vzVF0ndBpYex1saFa6sv0koEYQ-dncISKy0S20vQZjyfiSeNWZbVdQRo7G-tDUUzhHXPO-zclC5sF9-TDrmigTt5Ak_mCY1h1dB060qe0NWB99Wjn6c6TGbmPrx6SySxhCKqShm2tJuGHpMF-b6mEOXdwtiBsmS-UPpc92C8KCmBzC4g5_w7A9T-_JqM6s_UqhcFwnE0rZG8GgciIxO-VWa552y9dTMQpNiVkyQ3Fmy5l4Ev6967rFtmqcwRTPKfFoiCyriNi_WddZhYzjyewNPcHyXvr9BDwKkRiR_gaqgB3tbl2GNsDEy

200 OK
Length: 48847 (48K) [application/pdf]
Saving to: ‘state_of_the_union.pdf’

state_of_the_union. 100%[===================>]  47.70K  --.-KB/s    in 0s      

2023-06-08 00:33:07 (119 MB/s) - ‘state_of_the_union.pdf’ saved [48847/48847]



### Load PDFs

In [3]:
drop_pdf = cursor.query("DROP TABLE IF  EXISTS MyPDFs").execute()
load_pdf1 = cursor.load(file_regex="layout-parser-paper.pdf", format="PDF", table_name="MyPDFs").execute()
load_pdf1 = cursor.load(file_regex="state_of_the_union.pdf", format="PDF", table_name="MyPDFs").execute()

### Retrieve Text from Loaded PDFs

In [4]:
cursor.table("MyPDFs").df()

,mypdfs._row_id,mypdfs.name,mypdfs.page,mypdfs.paragraph,mypdfs.data
0,1,layout-parser-paper.pdf,1,1,LayoutParser: A Uniﬁed Toolkit for DeepLearnin...
1,1,layout-parser-paper.pdf,1,2,"Zejiang Shen1(�), Ruochen Zhang2, Melissa Dell..."
2,1,layout-parser-paper.pdf,1,3,1 Allen Institute for AIshannons@allenai.org2 ...
3,1,layout-parser-paper.pdf,1,4,Abstract. Recent advances in document image an...
4,1,layout-parser-paper.pdf,1,5,Keywords: Document Image Analysis · Deep Learn...
...,...,...,...,...,...
512,2,state_of_the_union.pdf,17,19,Now is our moment to meet and overcome the cha...
513,2,state_of_the_union.pdf,17,20,"And we will, as one people."
514,2,state_of_the_union.pdf,17,21,One America.
515,2,state_of_the_union.pdf,17,22,The United States of America.


### Create Sentence Transformer Feature Extractor UDF

In [5]:
udf_check = cursor.query(
            "DROP UDF IF  EXISTS SentenceTransformerFeatureExtractor"
        )
udf_check.execute()
udf = cursor.create_udf(
    "SentenceTransformerFeatureExtractor",
    True,
    "../evadb/udfs/sentence_transformer_feature_extractor.py",
)
udf.execute()

06-08-2023 00:33:12 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0082] UDF SentenceTransformerFeatureExtractor does not exist, therefore cannot be dropped.


### Create vector index for paragraphs of pdf

In [6]:
cursor.create_vector_index(
            "faiss_indexs",
            table_name="MyPDFs",
            expr="SentenceTransformerFeatureExtractor(data)",
            using="QDRANT",
        ).df()

,0
0,Index faiss_indexs successfully added to the d...


In [7]:
query = (
            cursor.table("MyPDFs")
            .order(
                """Similarity(
                    SentenceTransformerFeatureExtractor('When was the NATO created?'), SentenceTransformerFeatureExtractor(data)
                )"""
            )
            .limit(1)
            .select("data")
        ).df()
query

,mypdfs.data
0,"[38] Zhong, X., Tang, J., Yepes, A.J.: Publayn..."
